# 使用 Claude 构建内容审核过滤器

本指南将向您展示如何使用 Claude 为用户生成文本构建内容审核过滤器。关键思想是在提示词中直接定义审核规则和类别，从而实现轻松的自定义和实验。

## 基本方法

基本方法是向 Claude 提供一个提示词，描述您想要过滤的类别（例如"ALLOW"和"BLO CK"），以及每种类型内容的详细描述或示例。然后，您将待分类的用户生成文本作为提示词的一部分插入，并要求 Claude 根据提供的指导方针对其进行分类。

以下是一个提示词结构示例：

```text
您是一位内容审核专家，负责根据以下指导方针对用户生成文本进行分类：

BLOCK 类别：
- [应被阻止的内容描述或示例]

ALLOW 类别：
- [允许内容的描述或示例]

以下是待分类的用户生成文本：
<user_text>{{USER_TEXT}}</user_text>

根据上述指导方针将此文本分类为 ALLOW 或 BLOCK。不要返回其他内容。
```

要使用此方法，您需要将 `{{USER_TEXT}}` 替换为待分类的实际用户生成文本，然后使用 Claude API 发送提示词。Claude 的响应应该是"ALLOW"或"BLO CK"，表示应根据您提供的指导方针如何处理该文本。

## 示例用法

以下是一些示例 Python 代码，演示如何使用这种方法：

In [ ]:
%pip install anthropic

In [2]:
from anthropic import Anthropic

client = Anthropic()
MODEL_NAME = "claude-haiku-4-5"


def moderate_text(user_text, guidelines):
    prompt_template = """
    You are a content moderation expert tasked with categorizing user-generated text based on the following guidelines:

    {guidelines}

    Here is the user-generated text to categorize:
    <user_text>{user_text}</user_text>

    Based on the guidelines above, classify this text as either ALLOW or BLOCK. Return nothing else.
    """

    # Format the prompt with the user text
    prompt = prompt_template.format(user_text=user_text, guidelines=guidelines)

    # Send the prompt to Claude and get the response
    response = (
        client.messages.create(
            model=MODEL_NAME, max_tokens=10, messages=[{"role": "user", "content": prompt}]
        )
        .content[0]
        .text
    )

    return response

以下是如何使用此函数审核用户评论数组的示例：

In [3]:
example_guidelines = """BLOCK CATEGORY:
    - Promoting violence, illegal activities, or hate speech
    - Explicit sexual content
    - Harmful misinformation or conspiracy theories

    ALLOW CATEGORY:
    - Most other content is allowed, as long as it is not explicitly disallowed
"""

user_comments = [
    "This movie was great, I really enjoyed it. The main actor really killed it!",
    "Delete this post now or you better hide. I am coming after you and your family.",
    "Stay away from the 5G cellphones!! They are using 5G to control you.",
    "Thanks for the helpful information!",
]

for comment in user_comments:
    classification = moderate_text(comment, example_guidelines)
    print(f"Comment: {comment}\nClassification: {classification}\n")

Comment: This movie was great, I really enjoyed it. The main actor really killed it!
Classification: ALLOW

Comment: Delete this post now or you better hide. I am coming after you and your family.
Classification: BLOCK

Comment: Stay away from the 5G cellphones!! They are using 5G to control you.
Classification: BLOCK

Comment: Thanks for the helpful information!
Classification: ALLOW



## 自定义

这种方法的主要优势之一是您可以通过修改提示词中"BLO CK"和"ALLOW"类别提供的描述或示例来轻松自定义审核规则。这使您能够微调过滤以适应您的特定需求或偏好。

例如，如果您希望 Claude 审核一个过山车爱好者论坛并确保帖子保持主题相关，您可以相应地更新"ALLOW"和"BLO CK"类别描述：

In [4]:
rollercoaster_guidelines = """BLOCK CATEGORY:
- Content that is not related to rollercoasters, theme parks, or the amusement industry
- Explicit violence, hate speech, or illegal activities
- Spam, advertisements, or self-promotion

ALLOW CATEGORY:
- Discussions about rollercoaster designs, ride experiences, and park reviews
- Sharing news, rumors, or updates about new rollercoaster projects
- Respectful debates about the best rollercoasters, parks, or ride manufacturers
- Some mild profanity or crude language, as long as it is not directed at individuals
"""

post_titles = [
    "Top 10 Wildest Inversions on Steel Coasters",
    "My Review of the New RMC Raptor Coaster at Cedar Point",
    "Best Places to Buy Cheap Hiking Gear",
    "Rumor: Is Six Flags Planning a Giga Coaster for 2025?",
    "My Thoughts on the Latest Marvel Movie",
]

for title in post_titles:
    classification = moderate_text(title, rollercoaster_guidelines)
    print(f"Title: {title}\nClassification: {classification}\n")

Title: Top 10 Wildest Inversions on Steel Coasters
Classification: ALLOW

Title: My Review of the New RMC Raptor Coaster at Cedar Point
Classification: ALLOW

Title: Best Places to Buy Cheap Hiking Gear
Classification: BLOCK

Title: Rumor: Is Six Flags Planning a Giga Coaster for 2025?
Classification: ALLOW

Title: My Thoughts on the Latest Marvel Movie
Classification: BLOCK



## 使用思维链（CoT）提高性能

一种可以增强 Claude 内容审核能力的技术是"思维链"（CoT）提示。这种方法鼓励 Claude 将其推理过程分解为逐步的思维链，而不仅仅是提供最终输出。

要利用思维链进行审核，您可以修改提示词，明确指示 Claude 在 `<thinking>` 标签内将其过程分解为清晰的步骤。以下是一个示例：

In [8]:
cot_prompt = """You are a content moderation expert tasked with categorizing user-generated text based on the following guidelines:

BLOCK CATEGORY:
- Content that is not related to rollercoasters, theme parks, or the amusement industry
- Explicit violence, hate speech, or illegal activities
- Spam, advertisements, or self-promotion

ALLOW CATEGORY:
- Discussions about rollercoaster designs, ride experiences, and park reviews
- Sharing news, rumors, or updates about new rollercoaster projects
- Respectful debates about the best rollercoasters, parks, or ride manufacturers
- Some mild profanity or crude language, as long as it is not directed at individuals

First, inside of <thinking> tags, identify any potentially concerning aspects of the post based on the guidelines below and consider whether those aspects are serious enough to block the post or not. Finally, classify this text as either ALLOW or BLOCK inside <output> tags. Return nothing else.

Given those instructions, here is the post to categorize:

<user_post>{user_post}</user_post>"""

user_post = "Introducing my new band - Coaster Shredders. Check us out on YouTube!!"

response = (
    client.messages.create(
        model=MODEL_NAME,
        max_tokens=1000,
        messages=[{"role": "user", "content": cot_prompt.format(user_post=user_post)}],
    )
    .content[0]
    .text
)

print(response)

<thinking>
The post appears to be promoting a band rather than discussing rollercoasters, theme parks, or the amusement industry. This falls under the "spam, advertisements, or self-promotion" category, which is grounds for blocking the post.
</thinking>

<output>BLOCK</output>


## 使用示例提高性能

提高性能的另一种技术是通过在提示词中添加一些示例，为 Claude 提供一些初始训练数据或"少样本学习"，以便更好地理解所需的分类。这对于类别边界可能不完全明确或模棱两可的情况特别有帮助。以下是如何修改提示词模板以包含示例的示例：

In [9]:
examples_prompt = """You are a content moderation expert tasked with categorizing user-generated text based on the following guidelines:

BLOCK CATEGORY:
- Content that is not related to rollercoasters, theme parks, or the amusement industry
- Explicit violence, hate speech, or illegal activities
- Spam, advertisements, or self-promotion

ALLOW CATEGORY:
- Discussions about rollercoaster designs, ride experiences, and park reviews
- Sharing news, rumors, or updates about new rollercoaster projects
- Respectful debates about the best rollercoasters, parks, or ride manufacturers
- Some mild profanity or crude language, as long as it is not directed at individuals

Here are some examples:
<examples>
Text: I'm selling weight loss products, check my link to buy!
Category: BLOCK

Text: I hate my local park, the operations and customer service are terrible. I wish that place would just burn down.
Category: BLOCK

Text: Did anyone ride the new RMC raptor Trek Plummet 2 yet? I've heard it's insane!
Category: ALLOW

Text: Hercs > B&Ms. That's just facts, no cap! Arrow > Intamin for classic woodies too.
Category: ALLOW
</examples>

Given those examples, here is the user-generated text to categorize:
<user_text>{user_text}</user_text>

Based on the guidelines above, classify this text as either ALLOW or BLOCK. Return nothing else."""

user_post = "Why Boomerang Coasters Ain't It (Don't @ Me)"

response = (
    client.messages.create(
        model=MODEL_NAME,
        max_tokens=1000,
        messages=[{"role": "user", "content": examples_prompt.format(user_text=user_post)}],
    )
    .content[0]
    .text
)

print(response)

ALLOW
